In [4]:
import scipy.stats as scs
import pandas as pd
import numpy as np

Будем строить прогноз, исходя из нескольких показателей, которые хотим отслеживать и нескольких вариантов uplift.

Расчет подходит для AB теста с одним тестовым вариантом, с равным делением трафика 50% / 50%.

In [5]:
# Задаем параметры

# bcrs - словарь текущих показателей отслеживаемых показателей

bcrs = {
    'CR из пользователя в просмотр карточки товара': 0.22,
    'CR из пользователя в добавление в корзину': 0.05,
    'CR из пользователя в заказ': 0.01
}

# mde - oжидаемый uplift отслеживаемого показателя, относительная величина - на сколько процентов хотим увеличить наш показатель
mdes = [0.01, 0.02, 0.03, 0.05, 0.07, 0.10, 0.15, 0.20]

# количество пользователей в день
users_day = 10000

# мощность теста
power=0.8

# минимальный уровень стат.значимости
pvalue=0.05

In [6]:
def min_sample_size(bcr, mde, users_day, power, pvalue):
    """
    Функция возвращает минимальный размер выборки для настройки AB-теста
     Аргументы:
         bcr (float): базовая конверсия
         mde (float): минимальный uplift, который ожидаем получить
         users_day () - количество пользователей в день
         power (float): вероятность отклонения нулевой гипотезы, когда
         нулевая гипотеза неверна, обычно 0,8
         sig_level (float): уровень значимости часто обозначается как альфа,
         обычно 0,05
     Возвращает:
         датафрейм с прогнозом продолжительности AB теста по каждому показателю по каждому uplift
     Ссылки:
         Стэнфордская лекция о размерах выборки: http://statweb.stanford.edu/~susan/courses/s141/hopower.pdf
         Калькулятор Эвана Миллера для расчета продолжительности теста: https://www.evanmiller.org/ab-testing/sample-size.html
    """
    
    #################################################################
    # набор колонок для датафрейма имходя из нужных uplift
    columns = ['conversion_name','current_rate', 'users_day']
    for item in mdes:
        column_temp = 'uplift '+str(round(item*100, 2))+'%'
        columns.append(column_temp)
    result = pd.DataFrame(columns = columns)
    #################################################################
    
    marks = bcrs.keys()
    for item in marks:
        
        # массив значений для строки таблицы
        result_calculate = []
        
        # заполняем массив базовыми значениями
        cr_name = item
        bcr = bcrs[item]
        result_calculate.append(item)
        result_calculate.append(bcr)
        result_calculate.append(users_day)
        
        # считаем необходимое количество дней для каждого uplift
        for mde in mdes:
            column = 'uplift '+str(round(mde*100, 2))+'%'

            mde_relative = mde*bcr
            
            standard_norm = scs.norm(0, 1)

            Z_beta = standard_norm.ppf(power)

            Z_alpha = standard_norm.ppf(1-pvalue/2)

            pooled_prob = (bcr + bcr*mde_relative) / 2

            min_N = (2 * pooled_prob * (1 - pooled_prob) * (Z_beta + Z_alpha)**2 / mde_relative**2) * 2
            
            # необходимое количество дней
            calc_days = int(round(min_N/users_day))
            if calc_days == 0: count_days = 1
            else: count_days = calc_days
            result_calculate.append(count_days)
        
        # создаем датафрейм с результатами
        result_temp = pd.DataFrame(np.array([result_calculate]), columns = columns)

        result = result.append(result_temp, ignore_index=False)
    return result

In [7]:
calculate = min_sample_size(bcrs, mdes, users_day, power=0.8, pvalue=0.05)
calculate.to_csv('calculate_ab_test.csv')
calculate

,conversion_name,current_rate,users_day,uplift 1.0%,uplift 2.0%,uplift 3.0%,uplift 5.0%,uplift 7.0%,uplift 10.0%,uplift 15.0%,uplift 20.0%
0,CR из пользователя в просмотр карточки товара,0.22,10000,112,28,13,5,2,1,1,1
0,CR из пользователя в добавление в корзину,0.05,10000,599,151,67,24,13,6,3,2
0,CR из пользователя в заказ,0.01,10000,3124,785,350,127,66,33,15,9
